In [1]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

In [3]:
from graph_rag.rag_store import GraphRAGStore
from graph_rag.rag_extractor import GraphRAGExtractor
from graph_rag.rag_query_engine import GraphRAGQueryEngine
from llama_index.core import PropertyGraphIndex
from llama_index.llms.openai import OpenAI
import textwrap

import data_pull_and_prep.utils as utils
from graph_rag.utils import TranscriptionParser


In [4]:
transcription_with_char_timestamps = utils.import_pkl_file(project_root+"/data/audio_1/ivanka_trump_transcription_char_timestamps.pkl")

In [5]:
kg_extractor = GraphRAGExtractor(max_paths_per_chunk=4)

In [6]:
graph_store = GraphRAGStore(username="neo4j", password="neo4j_rishi", url="bolt://localhost:7687")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [7]:
transcription_parser = TranscriptionParser(transcription_with_char_timestamps)

index = PropertyGraphIndex(
    nodes=transcription_parser.get_nodes(),
    kg_extractors=[kg_extractor],
    property_graph_store=graph_store,
    show_progress=True,
)

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  6.67it/s]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [8]:
index.property_graph_store.build_communities()

In [9]:
default_llm = OpenAI(model="gpt-3.5-turbo")

query_engine = GraphRAGQueryEngine(
    graph_store=index.property_graph_store,
    llm=default_llm,
    index=index,
    similarity_top_k=10,
    num_children=10,
)

In [10]:
query_str = "What are some famous quotes mentioned in this podcast and who said them?"
response = query_engine.custom_query(query_str)
print("----------------------")
print(textwrap.fill(response, width=100))

----------------------
1. "When we are no longer able to change a situation, we are challenged to change ourselves." -
Victor Frankl 2. "The beauty of life is in the small details, not in big events." - Marcus Aurelius
3. "The way I see it, if you want the rainbow, you gotta put up with the rain." - Dolly Parton 4.
"The only way to make sense out of change is to plunge into it, move with it, and join the dance." -
Alan Watts


In [11]:
query_str = "What are Ivanka Trump's thoughts on music?"
response = query_engine.custom_query(query_str)
print("----------------------")
print(textwrap.fill(response, width=100))

----------------------
Based on the information provided, Ivanka Trump's specific thoughts on music are not explicitly
mentioned. However, her attendance at events like Adele's concert in Las Vegas suggests that she
appreciates music and likely finds it to be a source of enjoyment and entertainment in her life.
While her direct thoughts on music beyond this event are not detailed, her presence at such concerts
indicates a positive attitude towards music and live performances.
